In [1]:
import re
import requests
from bs4 import BeautifulSoup
import re  # 正则表达式，进行文字匹配`
import urllib.request, urllib.error  # 制定URL，获取网页数据
from bs4 import BeautifulSoup
from urllib.parse import quote
from openpyxl import Workbook
from openpyxl import load_workbook
import xlsxwriter
import csv
import pandas as pd

In [2]:
def GetInfo(html):
    if html is None:
        raise ValueError("HTML内容未定义")
    html = html.prettify()
    soup = BeautifulSoup(html,"html.parser",from_encoding="utf-8")
    title_tag = soup.find('a')
    title = title_tag.get_text(strip=True) if title_tag else "未找到标题"

    # 找到作者的<a>标签并提取文本
    author_tag = soup.find('a', rel="author")
    author = author_tag.get_text(strip=True) if author_tag else "未找到作者"
    comments_text= soup.find('dd', class_='comments')
    if comments_text:
        comments = comments_text.get_text(strip=True)
    else:
        comments=0 
    bookmarks= soup.find('dd', class_='bookmarks')
    if bookmarks:
        bookmarks = bookmarks.get_text(strip=True)  
    else:
        bookmarks=0    
    if soup.find('dd', class_='kudos'):
        kudos=soup.find('dd', class_='kudos').get_text(strip=True)
    else:
        kudos=0
    if soup.find('dd', class_='hits'):
        hits=soup.find('dd', class_='hits').get_text(strip=True)
    else:
        hits=0  
    stats = {
    "标题":title,
    "作者":author,
    "点赞":kudos,
    "点击":hits,
    "评论":comments,   
    "收藏":bookmarks   
    }
    return stats

In [3]:
# def SaveSheet(InfoDict,worksheet):
#     for row, items in enumerate(data.values()):
#         for col, item in enumerate(items):
#             sheet.cell(row=row+1, column=col+1, value=item)
#     # 保存Excel文件
#     workbook.save(file_path)
#     workbook = Workbook()
#     sheet = workbook.active  # 激活sheet
#     sheet.title = name  # 设置sheet名字
#     sheet.append(["标题", "作者", "点赞","点击", "评论", "收藏"])  # 插入标题

In [4]:
def askURL(url):
    head = {  # 模拟浏览器头部信息，向豆瓣服务器发送消息
        "User-Agent": "Mozilla / 5.0(Windows NT 10.0; Win64; x64) AppleWebKit / 537.36(KHTML, like Gecko) Chrome / 80.0.3987.122  Safari / 537.36"
    }
    # 用户代理，表示告诉豆瓣服务器，我们是什么类型的机器、浏览器（本质上是告诉浏览器，我们可以接收什么水平的文件内容）
    request = urllib.request.Request(url, headers=head)
    html = ""
    try:
        response = urllib.request.urlopen(request)
        html = response.read().decode("utf-8")
    except urllib.error.URLError as e:
        if hasattr(e, "code"):
            print(e.code)
        if hasattr(e, "reason"):
            print(e.reason)
    return html

In [5]:
def NumsOfPages(url):
    html=askURL(url)
    # 定义HTML内容
    soup = BeautifulSoup(html,"html.parser",from_encoding="utf-8")
    PagesInfo=soup.find('ol',class_="pagination actions")
    if PagesInfo is None:
        return "未找到分页信息"
    num=0
    for item in PagesInfo.find_all('li'):
        pages = item.find('a')
        pages = item.get_text(strip=True) if item else "未找到标题"
        if(pages.isdigit()):
            num=pages
    return int(num)

In [6]:
if __name__ == "__main__":
    
    chinese_str = input("请输入查询角色：")
#     file_path = 'output.xlsx'
#     # 创建一个Excel文件
#     workbook = xlsxwriter.Workbook(file_path)
#     # 添加一个工作表
#     sheet_name = chinese_str
#     if sheet_name not in workbook.sheetnames:
#         worksheet = workbook.create_sheet(title=sheet_name)
#     else:
#         worksheet = workbook[sheet_name]
    file_path=chinese_str+'.csv'
    
    encoded_url = quote(chinese_str)
    base_url='https://archiveofourown.org/works/search?'
    query_url='work_search%5Bquery%5D='+encoded_url
    url=f"{base_url}&{query_url}"
    
    
    num=NumsOfPages(url)
    num=int(num)
    begin_url='https://archiveofourown.org/works/search?'
    end_url='&work_search%5Bquery%5D='+encoded_url
    
    fieldnames = ["标题", "作者", "点赞","点击", "评论", "收藏"]
    
    with open(file_path,'w', newline='',encoding='utf-8-sig') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)       
        writer.writeheader()
        for page in range(1, num + 1):
            url = f"{begin_url}page={page}&{end_url}"
            html = askURL(url)  # 保存获取到的网页源码
            soup = BeautifulSoup(html, "html.parser")
            for item in soup.find_all('li', role="article"):
                singleInfo=GetInfo(item)
                writer.writerow(singleInfo)
                
        csv_file.close()


#             worksheet["标题"]=singleInfo["标题"]
#             worksheet["作者"]=singleInfo["作者"]
#             worksheet["点赞"]=singleInfo["点赞"]
#             worksheet["点击"]=singleInfo["点击"]
#             worksheet["评论"]=singleInfo["评论"]
#             worksheet["收藏"]=singleInfo["收藏"]

请输入查询角色：秦彻


C:\Users\a2693\Anaconda3\envs\pytorch\lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
